In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from data_structure.test import *

In [3]:
%%time
origin_result_df = deal_data()
# origin_result_df

Wall time: 3.6 s


In [4]:
hoRatio = 0.1
offset_day = 7
# offset_day_list = range(1, 40, 1)
k = 70
# k_list = range(2, 100, 1)

In [5]:
# %time
%time ret = guess_price(origin_result_df, hoRatio=hoRatio, offset_day=offset_day, k=k)
ret

Wall time: 5.43 s


(0.4666666666666667, 0.47058823529411764, 7, 70, 0.1)

In [6]:
%%time
# import time
# time.sleep(1)
from sklearn import neighbors
from sklearn import preprocessing
clf = neighbors.KNeighborsClassifier(
    n_neighbors=k, weights='uniform', algorithm='auto', leaf_size=1, p=2, metric='minkowski', metric_params=None, n_jobs=1)
print(clf)

KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=70, p=2,
           weights='uniform')
Wall time: 1.88 s


In [9]:
%%time

import itertools

# temp_df = origin_result_df.copy()
# temp_df = temp_df.reset_index(drop=False)
# temp_df['Close_change'] = '-'
# temp_df['inX_lable'] = 'CALCULATING'
# print(temp_df)

%time o_result_df = kNN.autoNorm(origin_result_df)
# print(result_df)
# %time X_normalized = preprocessing.normalize(origin_result_df, norm='l2')
# print(X_normalized)

result_list = []

offset_day_list = range(1, 15, 1)
k_list = range(2, 110, 1)
# for k in range(1, 110, 1):
for offset_day, k in itertools.product(offset_day_list, k_list):
    result_df = o_result_df.copy()
    result_df = result_df.reset_index(drop=False)
    result_df['Close_change'] = pd.np.nan
    # result_df['Close_change2'] = pd.np.nan

    # for i in range(0, len(result_df), 1):
    #     try:
    #         result_df.loc[i, 'Close_change'] = 'UP' if result_df.loc[i + offset_day, 'close'] > result_df.loc[i, 'close'] else 'DOWN'
    #         temp_df.loc[i, 'Close_change'] = result_df.loc[i, 'Close_change']
    #     except Exception as e:
    #         pass


    result_df['Close_change'] = result_df.close[:len(result_df)-offset_day] - result_df.close[offset_day:].values
    result_df['Close_change'] = result_df['Close_change'].apply(lambda x: 'UP' if x <=0 else 'DOWN')
    result_df.loc[len(result_df)-offset_day: ,'Close_change'] = pd.np.nan

    result_df.dropna(inplace=True)
    # print(result_df)

    numTestVecs = int(hoRatio * len(result_df))
    inX_label = result_df.loc[:, 'Close_change']
    train_label = inX_label.loc[:len(result_df)-numTestVecs]
    sample_label = inX_label.loc[len(result_df)-numTestVecs:]

    result_df = result_df.loc[:, ['close', 'volume', 'News_count', 'News_sentiment']]
    train_df = result_df.loc[:len(result_df)-numTestVecs]
    sample_df = result_df.loc[len(result_df)-numTestVecs:]

    clf = neighbors.KNeighborsClassifier(
        n_neighbors=k, weights='uniform', algorithm='auto', leaf_size=1, p=2, metric='minkowski', metric_params=None, n_jobs=1)
    clf.fit(train_df, train_label)
    # prediction = clf.predict([(0.04380037, 0.17151515, 0, 0)])
    # print(prediction)
    # prediction = clf.predict(sample_df)
    # print(prediction)
    score = clf.score(sample_df, sample_label)
    # print(score)
    result_list.append((score, k, offset_day, clf))

# print(sorted(result_list)[::-1][:10])
for item in sorted(result_list)[::-1][:10]:
    print(item)

Wall time: 143 ms
(0.71604938271604934, 2, 11, KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform'))
(0.68292682926829273, 20, 6, KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform'))
(0.67073170731707321, 18, 6, KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=18, p=2,
           weights='uniform'))
(0.66666666666666663, 22, 13, KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=22, p=2,
           weights='uniform'))
(0.66666666666666663, 18, 8, KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=18, p=2,
           weights='uniform'))
(0.666666666

In [10]:
%%time

for score, k, offset_day, clf in sorted(result_list)[::-1][:50]:
    result_df = origin_result_df.copy()
    
    result_df['inX_lable'] = o_result_df.apply(lambda x: clf.predict([x.values])[0], axis=1)

    result_df = result_df.reset_index(drop=False)
    result_df['Close_change'] = pd.np.nan
    result_df['Close_change'] = result_df.close[:len(result_df)-offset_day] - result_df.close[offset_day:].values
    result_df['Close_change'] = result_df['Close_change'].apply(lambda x: 'UP' if x <=0 else 'DOWN')
    result_df.loc[len(result_df)-offset_day: ,'Close_change'] = pd.np.nan
    
#     print(result_df)

    last_property = trade_strategy(result_df, offset_day, hoRatio)
    print(int(last_property), score, k, offset_day)

105000 0.716049382716 2 11
107688 0.682926829268 20 6
109708 0.670731707317 18 6
100794 0.666666666667 22 13
106164 0.666666666667 18 8
105876 0.666666666667 17 8
106512 0.658536585366 22 6
99544 0.654320987654 27 8
102523 0.654320987654 26 8
100923 0.654320987654 25 8
100970 0.654320987654 24 13
101621 0.654320987654 24 8
102800 0.654320987654 22 8
100524 0.654320987654 21 13
100547 0.654320987654 20 13
103550 0.654320987654 19 8
99055 0.654320987654 18 13
99359 0.654320987654 17 13
102616 0.654320987654 2 13
100768 0.641975308642 50 13
100768 0.641975308642 48 13
100017 0.641975308642 28 14
97405 0.641975308642 26 11
97689 0.641975308642 25 11
100391 0.641975308642 23 13
101032 0.641975308642 23 8
103514 0.641975308642 20 8
100221 0.641975308642 16 13
98532 0.641975308642 16 10
99724 0.641975308642 14 13
103932 0.641975308642 2 12
104782 0.634146341463 24 6
101916 0.634146341463 18 7
100682 0.62962962963 52 13
100682 0.62962962963 51 13
99500 0.62962962963 46 13
97304 0.62962962963 4